In [1]:
import pandas as pd
from fullcontact import FullContactClient
import numpy as np
from imblearn.over_sampling import SMOTENC
from helpers import Data
from urllib.request import urlopen, Request
import mysql.connector
from getpass import getpass
from mysql.connector import connect
from bs4 import BeautifulSoup
import requests
import re
from helpers import individualFeatures, Data, Scraping


w = pd.read_csv(r'C:\Users\aacjp\Sales-Scientist\datasets\check6.csv').drop(['Unnamed: 0'], axis='columns')
df = pd.read_csv(r'C:\Users\aacjp\Sales-Scientist\datasets\leadsML.csv').drop(['Unnamed: 0'], axis='columns')

In [ ]:
connection = connect(host='localhost', user='b7a35a7346aea6', password='Raptor//Kona9', database='leads')
cursor = connection.cursor()